# Hands On Astropy Tutorial
**Tutors:** Axel Donath and Johannes King

This hands-on tutorial gives an introduction and overview of the [Astropy](http://www.astropy.org/) Python package for astronomy.

![](astropy_banner_96.png)



## Preface
We recommend to follow this tutorial by **executing the code cells on your local machine**, along with the tutor. Every sub-topic we will cover in this tutorial will be concluded by a few **exercises with different levels of difficulty** (*easy*, *advanced* and *hard*). We will give you **~10-15 min to solve the exercises** and present a possible solution afterwards. In case we don't have time to show all the solutions or in the likely case that you don't manage to solve all of them: we will provide a **sample solution** for all exercises **after the course**.   

The estimated time for this tutorial is ~2 hours. Feel free to **interrupt at any time** to ask questions or **talk to the assisting tutors** when you encounter errors or the code doesn't work as expected.


We're happy to receive any feedback or questions on the tutorial via mail to *axel.donath@mpi-hd.mpg.de* or using the 
repository's [issue tracker](https://github.com/Asterics2020-Obelics/School2017/issues). 


## Overview

The material presented here is partly based on: 

- Astropy [documentation](http://docs.astropy.org/en/stable/index.html)
- Notebooks provided in the [Astropy Tutorials](https://github.com/astropy/astropy-tutorials) repository.
- And the [Astropy Introduction for Gammapy Users](https://github.com/gammapy/gammapy-extra/blob/master/notebooks/astropy_introduction.ipynb) notebook.


The Astropy package is structured into several submodules and we will cover (what we consider) the most important of them in the following order:

- [astropy.units](http://docs.astropy.org/en/stable/units/index.html) and in particular [astropy.units.Quantities](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html) to do astronomical calculations with units.

- [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) and in particular the classes [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) and [Angle](http://docs.astropy.org/en/stable/coordinates/angles.html) to handle astronomical sky positions, coordinate systems and coordinate transformations.

- [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) to open and write data files in [FITS format](https://fits.gsfc.nasa.gov/fits_documentation.html).

- [astropy.tables](http://docs.astropy.org/en/stable/table/index.html) and [Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html) class to handle astronomical data tables.

- (optional) Plotting of astronomical sky images with [astropy.visualization.wcsaxes](http://docs.astropy.org/en/stable/visualization/wcsaxes/)
#- (optional) Define regions in the sky with the [region](http://astropy-regions.readthedocs.io/en/latest/getting_started.html) package
- (optional) manipulate [Times and Dates](http://docs.astropy.org/en/stable/time/index.html)



## What is Astropy?


    "The Astropy Project is a community effort to develop a single core package for Astronomy in Python and foster     interoperability between Python astronomy packages."


[Astropy paper](http://adsabs.harvard.edu/abs/2013A%26A...558A..33A)

In addition to the Astropy core package there is an infrastructure of  [Astropy affiliated packages](http://www.astropy.org/affiliated/) dedicated to specific fields od astronomy or analysis tasks:




Let's start with the setup and check of the notebook:

# Setup

In [1]:
# to make plots appear inline in the notebook
%matplotlib inline  
import matplotlib.pyplot as plt

In [38]:
# If something below doesn't work, here's how you
# can check what version of Numpy and Astropy you have
# All examples should work with Astropy > 1.3 and Numpy > 1.11
import numpy as np
import astropy
print('numpy:', np.__version__)
print('astropy:', astropy.__version__)

numpy: 1.11.2
astropy: 1.3.2


# Astropy Units and Quantities

The [astropy.units]() subpackage provides functions and classes to handle physical quantities with units. 

This is the recommended way to import the `astropy.units` submodule: 

In [1]:
from astropy import units as u

`Quantities` are created by multiplying any number, list or array with a unit, where units are given as attributes on the `u` object:

In [36]:
distance = 1. * u.lightyear
print(distance)

1.0 lyr


Or using lists and arrays:

In [39]:
distances = [1, 3, 10] * u.lightyear
print(distances)

distances = np.array([1, 3, 10]) * u.lightyear
print(distances)

[  1.   3.  10.] lyr
[  1.   3.  10.] lyr


The quantity object has a value, which is a plain `numpy.ndarray`:

In [45]:
distances.value
type(distances.value)

numpy.ndarray

And a unit, which is represented by a `astropy.units.core.Unit` object:

In [46]:
distance.unit
type(distances.unit)

astropy.units.core.Unit

Probably the most used method on the `Quantity` object is the `.to()` method which allows to convert a quantity to different units:

In [55]:
distance.to('meter')

<Quantity 9460730472580800.0 m>

In [56]:
distance.to(u.parsec)

<Quantity 0.3066013937879527 pc>

Quantities can be combined with any arithmetical expression to derive other quantities and `astropy.units` will propagate
the units correctly:

In [30]:
speed_of_light = distance / u.year
print(speed_of_light.to('km/s'))

299792.458 km / s


In [ ]:
print(speed_of_light.to('angstrom/day'))

If a unit conversion is not possible an `UnitConversionError` will be raised:

In [64]:
speed_of_light.to('kg/s')

UnitConversionError: 'lyr / yr' (speed) and 'kg / s' are not convertible

For standardized unit systems such as 'si' or 'cgs' there are convenience attributes on the quantity object:

In [52]:
speed_of_light.si

<Quantity 299792458.0 m / s>

In [53]:
speed_of_light.cgs

<Quantity 29979245799.999992 cm / s>

Astropy provides a lot of builtin physical and astronomical constants in the [astropy.constants]() submodule:

In [51]:
from astropy.constants import c

print(c.to('km/s'))

299792.458 km / s


### A "real world" problem

When writing code that uses quantities we are typically bound to use it everywhere in the code. But often we'd like to interface with functions (e.g. from scipy or matplotlib) or other third-party code that doesn't know how to handle quantities. For matplotlib Astropy has a builtin support:

In [63]:
from astropy.

But in other cases there is only the option to convert the quantity to well defined units, strip off the unit and re-attach it after the computation:

In [73]:
emin, emax = [1, 10] * u.TeV
energies = np.logspace(np.log10(emin.value), np.log10(emax.value), 10) * emin.unit
print(energies)

[  1.           1.29154967   1.66810054   2.15443469   2.7825594
   3.59381366   4.64158883   5.9948425    7.74263683  10.        ] TeV


# Exercises

- (*easy*) Define a new unit called `"baro-meter"`, which is eqivalent to 25 cm and use it to measure the height of the empire state building (for other ways to measure the height of a building using a barometer see [barometer question on Wikipedia](https://en.wikipedia.org/wiki/Barometer_question)...)
- (*advanced*) 
- (*expert*) In gamma-ray astronomy the Crab Nebula is used as a standard candle to measure fluxes. Define a new equivalency that allows to convert

## See also
See also this [tutorial](http://www.astropy.org/astropy-tutorials/Quantities.html)

# Coordinates

In [4]:
from astropy.coordinates import SkyCoord

# Exercises

- (*easy*)
- (*advanced*)
- (*expert*)

# Fits I/O

The [flexible image transport system](https://fits.gsfc.nasa.gov/fits_documentation.html) format (FITS) is widely used data format for astronomical images and tables.



In [5]:
from astropy.io import fits

# Exercises

- (*easy*)
- (*advanced*)
- (*hard*)

# Tables

In [6]:
from astropy.table import Table

# Exercises

- (*easy*)
- (*advanced*)
- (*expert*)

# Plotting

# Exercises

- (*easy*)
- (*advanced*)
- (*expert*)

# Acknowledgements

This tutorial was supported by the H2020-Astronomy ESFRI and Research Infrastructure Cluster (Grant Agreement number: 653477).